In [ ]:
# Source: https://rpy2.github.io/doc/v3.4.x/html/robjects_rpackages.html#importing-arbitrary-r-code-as-a-package

# LATER SETUP DOCKER RPY2: https://github.com/rpy2/rpy2-docker/tree/master/jupyter_ds

In [1]:
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage

string = """
square <- function(x) {
    return(x^2)
}

cube <- function(x) {
    return(x^3)
}
"""

powerpack = SignatureTranslatedAnonymousPackage(string, "powerpack")

In [2]:
powerpack.square(2)

4.000000


In [4]:
string = """
run_rnaseq_analysis <- function(count_file, columns_to_delete = NULL, conditions) {
  # Load required libraries
  library(DESeq2)
  library(tidyverse)
  library(ggplot2)
  library(pheatmap)
  library(org.Mm.eg.db)  # Mouse database
  library(clusterProfiler)
  library(biomaRt)
  
  library(here)
  source(here("scripts", "convert_gene_names.R"))
  
  # Read count data
  counts <- read.table(count_file, header = TRUE, row.names = 1)
  
  # Delete specified columns if provided
  if (!is.null(columns_to_delete)) {
    if (is.numeric(columns_to_delete)) {
      # If columns_to_delete is a numeric index
      counts <- counts[, -columns_to_delete, drop = FALSE]
    } else {
      # If columns_to_delete is a vector of column names
      counts <- subset(counts, select = -all_of(columns_to_delete))
    }
  }
  
  # Create metadata
  if (length(conditions) != ncol(counts)) {
    stop("The length of 'conditions' must match the number of columns in the count data.")
  }
  
  coldata <- data.frame(
    row.names = colnames(counts),
    condition = factor(conditions)
  )
  
  # Create DESeq2 object
  dds <- DESeqDataSetFromMatrix(
    countData = counts,
    colData = coldata,
    design = ~ condition
  )
  
  # Pre-filtering: keep only rows with at least 10 reads total
  keep <- rowSums(counts(dds)) >= 10
  dds <- dds[keep,]
  
  # Run DESeq2 analysis
  dds <- DESeq(dds)
  
  # Get results
  res <- results(dds, alpha = 0.05, pAdjustMethod = "BH")
  res_ordered <- res[order(res$padj),]
  res_df <- as.data.frame(res_ordered)
  res_df$gene <- rownames(res_df)
  
  # Convert Ensembl IDs to gene symbols and names
  res_df <- convert_ensembl_to_symbol(res_df, "gene")
  
  # Write results to file with gene names
  write.csv(res_df, "deseq2_results_BH_with_names.csv")
  
  # Get significant genes
  sig_genes <- subset(res_df, padj < 0.05)
  write.csv(sig_genes, "significant_genes_with_names.csv")
  
  # GO Pathway Analysis
  sig_genes_list <- sig_genes$gene[!is.na(sig_genes$gene)]
  
  if (length(sig_genes_list) > 0) {
    go_enrichment <- enrichGO(
      gene = sig_genes_list,
      OrgDb = org.Mm.eg.db,
      keyType = "ENSEMBL",
      ont = "BP",  # Biological Process
      pAdjustMethod = "BH",
      pvalueCutoff = 0.05
    )
    
    if (!is.null(go_enrichment) && nrow(as.data.frame(go_enrichment)) > 0) {
      write.csv(as.data.frame(go_enrichment), "go_enrichment_results.csv")
      
      # Plot top GO terms
      png("go_dotplot.png", width = 800, height = 600)
      print(dotplot(go_enrichment, showCategory = 20))
      dev.off()
      
      # Create simplified GO network plot
      if (requireNamespace("enrichplot", quietly = TRUE)) {
        png("go_network.png", width = 1000, height = 800)
        print(enrichplot::emapplot(enrichplot::pairwise_termsim(go_enrichment)))
        dev.off()
      }
    }
  }
  
  # Volcano plot with gene symbols
  ggplot(res_df, aes(x = log2FoldChange, y = -log10(pvalue))) +
    geom_point(aes(color = padj < 0.05), size = 1) +
    scale_color_manual(values = c("grey", "red")) +
    labs(title = "Volcano Plot",
         x = "Log2 Fold Change",
         y = "-Log10 P-value") +
    theme_minimal() +
    theme(legend.position = "none")
  ggsave("volcano_plot.png")
  
  # Sample distance heatmap
  vsd <- vst(dds, blind = FALSE)
  sampleDists <- dist(t(assay(vsd)))
  sampleDistMatrix <- as.matrix(sampleDists)
  pheatmap(sampleDistMatrix,
           clustering_distance_rows = sampleDists,
           clustering_distance_cols = sampleDists,
           main = "Sample Distance Matrix",
           file = "sample_distance_heatmap.png")
  
  # PCA plot
  pcaData <- plotPCA(vsd, intgroup = "condition", returnData = TRUE)
  percentVar <- round(100 * attr(pcaData, "percentVar"))
  ggplot(pcaData, aes(x = PC1, y = PC2, color = condition)) +
    geom_point(size = 3) +
    xlab(paste0("PC1: ", percentVar[1], "% variance")) +
    ylab(paste0("PC2: ", percentVar[2], "% variance")) +
    ggtitle("PCA Plot") +
    theme_minimal()
  ggsave("pca_plot.png")
  
  # MA plot
  png("ma_plot.png")
  plotMA(res, ylim = c(-5, 5))
  dev.off()
  
  # Get top differentially expressed genes
  topGenes <- head(res_df[order(res_df$padj),], 20)
  write.csv(topGenes, "top_20_DEGs.csv")
  
  # Generate expression heatmap for top genes using gene names
  valid_rows <- rownames(topGenes) %in% rownames(vsd)
  if (sum(valid_rows) > 0) {
    mat <- assay(vsd)[rownames(topGenes)[valid_rows],]
    mat <- mat - rowMeans(mat)
    display_names <- topGenes$gene_symbol
    display_names[is.na(display_names)] <- rownames(topGenes)[is.na(display_names)]
    rownames(mat) <- display_names[valid_rows]
    pheatmap(mat,
             main = "Top 20 Differentially Expressed Genes",
             show_rownames = TRUE,
             scale = "row",
             file = "top_genes_heatmap.png")
  }
}
"""

rnaseq = SignatureTranslatedAnonymousPackage(string, "rnaseq")


In [8]:
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import StrVector

utils = importr('utils')
utils.install_packages(StrVector(["tidyverse", "DESeq2", "pheatmap", "org.Mm.eg.db", "clusterProfiler", "biomaRt"]))

--- Please select a CRAN mirror for use in this session ---


R[write to console]: also installing the dependencies ‘haven’, ‘readxl’


R[write to console]: trying URL 'https://cran.case.edu/src/contrib/haven_2.5.4.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 313332 bytes (305 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console

Found pkg-config cflags and libs!
Using PKG_CFLAGS=
Using PKG_LIBS=-lz
x86_64-conda-linux-gnu-cc -I"/home/ztba231/miniconda3/envs/bioenv/lib/R/include" -DNDEBUG  -I'/home/ztba231/miniconda3/envs/bioenv/lib/R/library/cpp11/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/ztba231/miniconda3/envs/bioenv/include -I/home/ztba231/miniconda3/envs/bioenv/include -Wl,-rpath-link,/home/ztba231/miniconda3/envs/bioenv/lib   -Ireadstat -DHAVE_ZLIB -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/ztba231/miniconda3/envs/bioenv/include -fdebug-prefix-map=/workspace/croot/r-base_1695428141831/work=/usr/local/src/conda/r-base-4.3.1 -fdebug-prefix-map=/home/ztba231/miniconda3/envs/bioenv=/usr/local/src/conda-prefix  -c tagged_na.c -o tagged_na.o


** libs
using C compiler: ‘x86_64-conda-linux-gnu-cc (Anaconda gcc) 11.2.0’
using C++ compiler: ‘x86_64-conda-linux-gnu-c++ (Anaconda gcc) 11.2.0’


x86_64-conda-linux-gnu-cc -I"/home/ztba231/miniconda3/envs/bioenv/lib/R/include" -DNDEBUG  -I'/home/ztba231/miniconda3/envs/bioenv/lib/R/library/cpp11/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/ztba231/miniconda3/envs/bioenv/include -I/home/ztba231/miniconda3/envs/bioenv/include -Wl,-rpath-link,/home/ztba231/miniconda3/envs/bioenv/lib   -Ireadstat -DHAVE_ZLIB -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/ztba231/miniconda3/envs/bioenv/include -fdebug-prefix-map=/workspace/croot/r-base_1695428141831/work=/usr/local/src/conda/r-base-4.3.1 -fdebug-prefix-map=/home/ztba231/miniconda3/envs/bioenv=/usr/local/src/conda-prefix  -c readstat/readstat_parser.c -o readstat/readstat_parser.o
x86_64-conda-linux-gnu-cc -I"/home/ztba231/miniconda3/envs/bioenv/lib/R/include" -DNDEBUG  -I'/home/ztba231/miniconda3/envs/bioenv/lib/R/library/cpp11/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /hom

...

x86_64-conda-linux-gnu-c++ -std=gnu++17 -I"/home/ztba231/miniconda3/envs/bioenv/lib/R/include" -DNDEBUG -Iunix -I. -I'/home/ztba231/miniconda3/envs/bioenv/lib/R/library/cpp11/include' -I'/home/ztba231/miniconda3/envs/bioenv/lib/R/library/progress/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/ztba231/miniconda3/envs/bioenv/include -I/home/ztba231/miniconda3/envs/bioenv/include -Wl,-rpath-link,/home/ztba231/miniconda3/envs/bioenv/lib   -fvisibility=hidden -fpic  -fvisibility-inlines-hidden  -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/ztba231/miniconda3/envs/bioenv/include -fdebug-prefix-map=/workspace/croot/r-base_1695428141831/work=/usr/local/src/conda/r-base-4.3.1 -fdebug-prefix-map=/home/ztba231/miniconda3/envs/bioenv=/usr/local/src/conda-prefix  -c cpp11.cpp -o cpp11.o
x86_64-conda-linux-gnu-c++ -std=gnu++17 -I"/home/ztba231/miniconda3/envs/bioenv/lib/R/include" -DNDEB

installing to /home/ztba231/miniconda3/envs/bioenv/lib/R/library/00LOCK-readxl/00new/readxl/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
ERROR: loading failed
* removing ‘/home/ztba231/miniconda3/envs/bioenv/lib/R/library/readxl’


Error: package or namespace load failed for ‘readxl’ in dyn.load(file, DLLpath = DLLpath, ...):
 unable to load shared object '/home/ztba231/miniconda3/envs/bioenv/lib/R/library/00LOCK-readxl/00new/readxl/libs/readxl.so':
  /home/ztba231/miniconda3/envs/bioenv/lib/R/library/00LOCK-readxl/00new/readxl/libs/readxl.so: undefined symbol: libiconv
Error: loading failed
Execution halted


* installing *source* package ‘pheatmap’ ...
** package ‘pheatmap’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (pheatmap)
ERROR: dependencies ‘haven’, ‘readxl’ are not available for package ‘tidyverse’
* removing ‘/home/ztba231/miniconda3/envs/bioenv/lib/R/library/tidyverse’
R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/tmp/Rtmpxy7FQr/downloaded_packages’
R[write to console]: 
R[write to console]: 

R[write to console]: Updating HTML index of packages in '.Library'

R[write to console]: Making 'packages.html' ...
R[write to console]:  done



<rpy2.rinterface_lib.sexp.NULLType object at 0x7f4018f58cc0> [RTYPES.NILSXP]

In [9]:
rnaseq.run_rnaseq_analysis(
  count_file = "C:/Users/ztba231/Documents/genes.readcount.cntlD14.txt",
  columns_to_delete = ["C_3_280N", "C_4_280N"],  # Python list
  conditions = ["control", "control", "treatment", "treatment"]  # Python list
)

R[write to console]: Error in library(tidyverse) : there is no package called ‘tidyverse’

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: packages ‘DESeq2’, ‘org.Mm.eg.db’, ‘clusterProfiler’, ‘biomaRt’ are not available for this version of R

Versions of these packages for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages 

R[write to console]: 2: 
R[write to console]: In (function (pkgs, lib, repos = getOption("repos"), contriburl = contrib.url(repos,  :
R[write to console]: 
 
R[write to console]:  installation of package ‘haven’ had non-zero exit status

R[write to console]: 3: 
R[write to console]: In (function (pkgs, lib, repos = getOption("repos"), contriburl = contrib.url(repos,  :
R[write to console]: 
 
R[write to console]:  installation of package ‘readxl’ had non-zero exit status

R[write to console]: 4: 
R[write to console]: In

RRuntimeError: Error in library(tidyverse) : there is no package called ‘tidyverse’
